In [4]:
import pandas as pd
import numpy as np
import matplotlib as mp

In [5]:
events = pd.read_csv('../data/events.csv.gzip',compression='gzip', dtype='object')
events.describe().transpose()

,count,unique,top,freq
date,2494423,2488829,2019-03-12 14:36:58.017,3
event_id,2494423,568,22,618228
ref_type,2494423,2,1891515180541284343,1882743
ref_hash,2494423,196049,7823950631004872496,2913
application_id,2494423,269,66,325696
attributed,2494423,2,False,2489324
device_countrycode,2494423,1,6333597102633388268,2494423
device_os_version,1022066,81,4.3537503246500966e+18,277691
device_brand,1164963,250,3.083058605577787e+17,579861
device_model,2406456,2624,2.33194661091417e+18,210697


**device_country_code** es unico por lo que no aporta informacion para segmentar los datos.
**event_uuid** es clave unica para cada registro

In [3]:
events.shape

(2494423, 22)

In [4]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2494423 entries, 0 to 2494422
Data columns (total 22 columns):
date                  object
event_id              object
ref_type              object
ref_hash              object
application_id        object
attributed            object
device_countrycode    object
device_os_version     object
device_brand          object
device_model          object
device_city           object
session_user_agent    object
trans_id              object
user_agent            object
event_uuid            object
carrier               object
kind                  object
device_os             object
wifi                  object
connection_type       object
ip_address            object
device_language       object
dtypes: object(22)
memory usage: 418.7+ MB


In [5]:
events.head()

,date,event_id,ref_type,ref_hash,application_id,attributed,device_countrycode,device_os_version,device_brand,device_model,...,trans_id,user_agent,event_uuid,carrier,kind,device_os,wifi,connection_type,ip_address,device_language
0,2019-03-05 00:09:36.966,0,1891515180541284343,2688759737656491380,38,False,6333597102633388268,5.908703034097298e+17,NaN,5.990116681709081e+18,...,NaN,NaN,a9c0b263-acb2-4577-92c5-cbde5d7a5db1,2.2481569642543155e+17,5.516623012570946e+18,7.531669329342818e+18,NaN,Cable/DSL,7858558567428669000,4.077062198952597e+17
1,2019-03-05 00:09:38.920,1,1891515180541284343,2688759737656491380,38,False,6333597102633388268,5.908703034097298e+17,NaN,5.990116681709081e+18,...,NaN,NaN,1cd98205-0d97-4ec2-a019-667997dbfe7a,2.2481569642543155e+17,9.977660181316936e+17,7.531669329342818e+18,NaN,Cable/DSL,7858558567428669000,4.077062198952597e+17
2,2019-03-05 00:09:26.195,0,1891515180541284343,2688759737656491380,38,False,6333597102633388268,5.908703034097298e+17,NaN,5.990116681709081e+18,...,NaN,NaN,f02e2924-21ae-492b-b625-9021ae0a4eca,2.2481569642543155e+17,5.516623012570946e+18,7.531669329342818e+18,NaN,Cable/DSL,7858558567428669000,4.077062198952597e+17
3,2019-03-05 00:09:31.107,2,1891515180541284343,2688759737656491380,38,False,6333597102633388268,5.908703034097298e+17,NaN,5.990116681709081e+18,...,NaN,NaN,a813cf45-a36e-4668-85e2-5395f1564e98,2.2481569642543155e+17,8.561152834701074e+18,7.531669329342818e+18,NaN,Cable/DSL,6324037615828123965,4.077062198952597e+17
4,2019-03-09 21:00:36.585,3,1891515180541284343,2635154697734164782,38,False,6333597102633388268,7.391843585977942e+18,NaN,5.960895846746644e+18,...,NaN,NaN,63a4f0aa-e147-469f-8c55-4ca4f8d0e310,2.2481569642543155e+17,8.731901657619553e+17,7.531669329342818e+18,NaN,Cable/DSL,2894495631302821483,3.3013777759776993e+18
